In [ ]:
from load_dataset import paths, load_helpers
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.data import AUTOTUNE
import matplotlib.pyplot as plt
import os
import io

In [ ]:
args = {"dataset": "", "augment":False, "type":"layers"}  # type = ["layers", "ops"]
ValidAudios = ["*.wav"]

In [ ]:
# set the batch size
BATCH_SIZE = 8
# grabs all image paths
audioPaths = list(paths.list_audios(args["dataset"]))
# build our dataset and data input pipeline
print("[INFO] loading the dataset...")
ds = tf.data.Dataset.from_tensor_slices(audioPaths)
ds = (ds
	.shuffle(len(audioPaths), seed=42)
	.map(load_helpers.load_images, num_parallel_calls=AUTOTUNE)
	.cache()
	.batch(BATCH_SIZE)
    )

# check if we should apply data augmentation
if args["augment"]:
	# check if we will be using layers to perform data augmentation
	if args["type"] == "layers":
		# initialize our sequential data augmentation pipeline
		aug = tf.keras.Sequential([
			preprocessing.RandomFlip("horizontal_and_vertical"),
			preprocessing.RandomZoom(
				height_factor=(-0.05, -0.15),
				width_factor=(-0.05, -0.15)),
			preprocessing.RandomRotation(0.3)
		])
		# add data augmentation to our data input pipeline
		ds = (ds
			.map(lambda x, y: augment_using_layers(x, y, aug),
				num_parallel_calls=AUTOTUNE)
		)
	# otherwise, we will be using TensorFlow image operations to
	# perform data augmentation
	else:
		# add data augmentation to our data input pipeline
		ds = (ds
			.map(augment_using_ops, num_parallel_calls=AUTOTUNE)
		)

# complete our data input pipeline
ds = (ds
	.prefetch(AUTOTUNE)
)
# grab a batch of data from our dataset
batch = next(iter(ds))

# initialize a figure
print("[INFO] visualizing the first batch of the dataset...")
title = "With data augmentation {}".format(
	"applied ({})".format(args["type"]) if args["augment"] else \
	"not applied")
fig = plt.figure(figsize=(BATCH_SIZE, BATCH_SIZE))
fig.suptitle(title)
# loop over the batch size
for i in range(0, BATCH_SIZE):
	# grab the image and label from the batch
	(image, label) = (batch[0][i], batch[1][i])
	# create a subplot and plot the image and label
	ax = plt.subplot(2, 4, i + 1)
	plt.imshow(image.numpy())
	plt.title(label.numpy().decode("UTF-8"))
	plt.axis("off")
# show the plot
plt.tight_layout()
plt.show()

In [1]:
import tensorflow_io as tfio
filepath = "4persons/a_1.wav"
audio = tfio.audio.AudioIOTensor(filepath)


<AudioIOTensor: shape=[53476     1], dtype=<dtype: 'int16'>, rate=16000>


In [7]:
print(audio.to_tensor().numpy().shape)

(53476, 1)
